In [1]:
import numpy as np
import sympy as sym
import numba
import pydae.build as db

### Electromechanical differential equations
\begin{eqnarray}
f_1 &=& \dot \delta = \Omega_b \left( \omega - \omega_s  \right) \\
f_2 &=& \dot \omega = \frac{1}{2H} \left( p_m - p_e - D   \left( \omega - \omega_s \right) \right)   
\end{eqnarray}

### Electric rotor differential equations
\begin{eqnarray}
f_3 &=& \dot e_q' =  \frac{1}{T'_{d0}} \left( -e'_q - \left(X_d - X'_d \right) i_d + v_f^\star \right) \\
f_4 &=& \dot e'_d = \frac{1}{T'_{q0}} \left( -e'_d - \left(X_q - X'_q \right) i_q \right)       
\end{eqnarray}


### Park transform
\begin{eqnarray}
g_1 &=&-v_d + v_t \sin\left(\delta - \theta_t\right) \\
g_2 &=&-v_q + v_t \cos\left(\delta - \theta_t\right) 
\end{eqnarray}

### Stator equations
\begin{eqnarray}
g_3 &=& v_q + R_a i_q + X'_d i_d - e'_q\\
g_4 &=& v_d + R_a i_d - X'_q i_q - e'_d\\
\end{eqnarray}

### Powers
\begin{eqnarray}
g_5 &=& -p_e + \left( v_q + R_a i_q  \right) i_q + \left( v_d + R_a i_d  \right) i_d \\
g_6 &=&  i_d v_d + i_q v_q - p_t \\
g_7 &=&  i_d v_q - i_q v_d - q_t
\end{eqnarray}

### Network equations
\begin{eqnarray}
g_8 &=&   p_t - \left(v_t V_0 \sin\left(\theta_t - \Theta_0\right)\right)/X_l\\
g_9 &=&   q_t + \left(v_t V_0 \cos\left(\theta_t - \Theta_0\right)\right)/X_l - v_t^2/X_l 
\end{eqnarray}    



## System definition 

In [1]:
params_dict = {'X_d':1.81,'X1d':0.3, 'T1d0':8.0,  # synnchronous machine d-axis parameters
               'X_q':1.76,'X1q':0.65,'T1q0':1.0,  # synnchronous machine q-axis parameters
               'R_a':0.003,
               'X_l': 0.02, 
               'H':3.5,'D':0.0,
               'Omega_b':2*np.pi*50,'omega_s':1.0,
               'V_0':1.0,'Theta_0':0.0}


u_ini_dict = {'p_m':0.8,'v_f':1.0}  # for the initialization problem
u_run_dict = {'p_m':0.8,'v_f':1.0}  # for the running problem (here initialization and running problem are the same)


x_list = ['delta','omega','e1q','e1d']    # dynamic states
y_ini_list = ['i_d','i_q','p_t','q_t','v_t','theta_t']   
y_run_list = ['i_d','i_q','p_t','q_t','v_t','theta_t']   

sys_vars = {'params':params_dict,
            'u_list':u_run_dict,
            'x_list':x_list,
            'y_list':y_run_list}

exec(db.sym_gen_str())  # exec to generate the required symbolic varables and constants

NameError: name 'np' is not defined

In [ ]:
# auxiliar equations
v_d = v_t*sin(delta - theta_t)  # park
v_q = v_t*cos(delta - theta_t)  # park

p_e = i_d*(v_d + R_a*i_d) + i_q*(v_q + R_a*i_q) # electromagnetic power

# dynamic equations
ddelta = Omega_b*(omega - omega_s) - K_delta*delta  # load angle
domega = 1/(2*H)*(p_m - p_e - D*(omega - omega_s)) # speed
de1q = 1/T1d0*(-e1q - (X_d - X1d)*i_d + v_f)
de1d = 1/T1q0*(-e1d + (X_q - X1q)*i_q)

# algrbraic equations
g_1 = v_q + R_a*i_q + X1d*i_d - e1q # stator
g_2 = v_d + R_a*i_d - X1q*i_q - e1d # stator
g_3 = i_d*v_d + i_q*v_q - p_t # active power 
g_4 = i_d*v_q - i_q*v_d - q_t # reactive power
g_5 = p_t - (v_t*V_0*sin(theta_t - Theta_0))/X_l  # network equation (p)
g_6 = q_t + (v_t*V_0*cos(theta_t - Theta_0))/X_l - v_t**2/X_l  # network equation (q)                

In [2]:
sys = {'name':'smib_milano_ex8p1_4ord',
       'params_dict':params,
       'f_list':[ddelta,domega,de1q,de1d],
       'g_list':[g_1,g_2,g_3,g_4,g_5,g_6],
       'x_list':x_list,
       'y_ini_list':y_ini_list,
       'y_run_list':y_run_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_ini_dict,
       'h_dict':{'p_m':p_m,'p_e':p_e, 'v_f':v_f}}

sys = db.system(sys)
db.sys2num(sys)

NameError: name 'params' is not defined